In [1]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from pymongo import MongoClient
import pandas as pd
import numpy as np
import src.pipe as pipe

In [2]:
db = MongoClient().billboard
data = [i['lyrics'] for i in db.lyrics.find()][:10]

In [7]:
data[5]

"When the visions around you\nBring tears to your eyes\nAnd all that surrounds you\nAre secrets and lies\nI'll be your strength\nI'll give you hope\nKeeping your faith when it's gone\nThe one you should call\nWas standing there all along\n\nAnd I will take you in my arms\nAnd hold you right where you belong\nTil' the day my life is through\nThis I promise you\nThis I promise you\n\nI've loved you forever\nIn lifetimes before\nAnd I promise you never\nWill you hurt anymore\nI give you my word\nI give you my heart\nThis is a battle we've won\nAnd with this vow\nForever has now begun\n\nJust close your eyes (close your eyes)\nEach loving day\nI know this feeling won't go away (no)\nTill the day my life is through\nThis I promise you..\nThis I promise you..\n\nOver and over I fall\nWhen I hear you call\nWithout you in my life, baby\nI just wouldn't be living at all\n\nAnd I will take you in my arms\nAnd hold you right where you belong\nTil' the day my life is through\nThis I promise you\nJ

In [45]:
doc = nlp(data[5])

In [48]:
type(doc[0])

spacy.tokens.token.Token

In [46]:
for token in doc:
    print(f'{token.text:10} {token.pos_:10} {token.lemma_:10} {token.is_stop:6}')

When       ADV        when            1
the        DET        the             1
visions    NOUN       vision          0
around     ADP        around          1
you        PRON       -PRON-          1

          SPACE      
               0
Bring      VERB       bring           0
tears      NOUN       tear            0
to         ADP        to              1
your       DET        -PRON-          1
eyes       NOUN       eye             0

          SPACE      
               0
And        CCONJ      and             1
all        DET        all             1
that       DET        that            1
surrounds  VERB       surround        0
you        PRON       -PRON-          1

          SPACE      
               0
Are        AUX        be              1
secrets    NOUN       secret          0
and        CCONJ      and             1
lies       NOUN       lie             0

          SPACE      
               0
I          PRON       -PRON-          1
'll        VERB       will            1


In [11]:
for token in doc:
    print(f'{token.text:10} {token.dep_}')

When       advmod
the        det
visions    nsubj
around     prep
you        pobj

          
Bring      ROOT
tears      dobj
to         prep
your       poss
eyes       pobj

          
And        cc
all        nsubj
that       nsubj
surrounds  relcl
you        dobj

          
Are        ROOT
secrets    attr
and        cc
lies       conj

          
I          nsubj
'll        aux
be         ROOT
your       poss
strength   attr

          
I          nsubj
'll        aux
give       ROOT
you        dative
hope       dobj

          
Keeping    ROOT
your       poss
faith      dobj
when       advmod
it         nsubjpass
's         auxpass
gone       advcl

          
The        det
one        nsubj
you        nsubj
should     aux
call       relcl

          
Was        aux
standing   ROOT
there      advmod
all        advmod
along      prep


         
And        cc
I          nsubj
will       aux
take       ROOT
you        dobj
in         prep
my         poss
arms       pobj

          


In [16]:
displacy.render(doc)

In [25]:
for i in doc[0].rights:

In [26]:
doc.ents

(one, Til, Forever, Til, Ooh)

In [44]:
doc.ents[2]

NameError: name 'doc' is not defined

In [28]:
spacy.explain('ORG')

'Companies, agencies, institutions, etc.'

In [29]:
displacy.render(doc, style='ent')

In [3]:
bbd = pipe.BillboardData()
bbd.load()

In [4]:
lyrics = db.lyrics.find({'_id':{'$in':list(bbd.df.track_id.values)}})

In [5]:
ldf = pd.DataFrame([[l['_id'], l['lyrics']] for l in lyrics], 
    columns=['track_id', 'text'])

In [6]:
ldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14265 entries, 0 to 14264
Data columns (total 2 columns):
track_id    14265 non-null object
text        14265 non-null object
dtypes: object(2)
memory usage: 223.0+ KB


In [7]:
ldf.head()

,track_id,text
0,000U7VJzLWjyk6J8lFqnMZ,"So so, oh for real? Oh f'real?\nHell yeah man\..."
1,000xQL6tZNLJzIrtIgxqSl,"Just stop lookin' for love\nGirl, you're young..."
2,003FTlCpBTM4eSqYSWPv4H,Days swiftly come and go\nI'm dreaming of her\...
3,003eoIwxETJujVWmNFMoZy,"You're on your own, kid\nYou are\n\nMake my wa..."
4,005lwxGU1tms6HGELIcUv9,This was never the way I planned\nNot my inten...


In [8]:
df = bbd.df.copy()

In [9]:
df = df.merge(ldf, how='right', on='track_id')

In [10]:
print(df.shape)
df = df.sample(1000)
print(df.shape)

(14305, 39)
(1000, 39)


In [11]:
bbmask = ~df.obj_id.isna()

In [12]:
df['spacy_doc'] = list(nlp.pipe(df.text))

MemoryError: 

In [ ]:
bb_adj = [token.text.lower() for doc in df[bbmask].spacy_doc 
          for token in doc if token.pos_=='ADJ']

In [ ]:
nb_adj = [token.text.lower() for doc in df[~bbmask].spacy_doc 
          for token in doc if token.pos_=='ADJ']

In [ ]:
from collections import Counter

In [ ]:
Counter(bb_adj).most_common(10)

In [ ]:
Counter(nb_adj).most_common(10)

In [14]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f8bbd640f98>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f8bbd521588>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f8bbd5215e8>)]